In [13]:
# Importing dependencies
import os
import glob
import pandas as pd

In [14]:
# Set working dir
os.chdir("/Users/williamforsyth/Documents/uc_davis/Homework_Repos/tableau-citi-bike-analysis/data/raw_data")

In [15]:
# Find all csf files in working dir, using global pattern matching
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [16]:
# combining data from csv files into list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [17]:
# exporting list of all csv data to a csv file
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [18]:
# a quick check of the combined csv
combined_csv.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1062,2020-06-01 00:00:03.3720,2020-06-01 00:17:46.2080,3419.0,Douglass St & 4 Ave,40.679279,-73.981540,3419.0,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,1997,2
1,3810,2020-06-01 00:00:03.5530,2020-06-01 01:03:33.9360,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336.0,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,1969,0
2,1017,2020-06-01 00:00:09.6140,2020-06-01 00:17:06.8330,389.0,Broadway & Berry St,40.710446,-73.965251,3562.0,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,1988,2
3,226,2020-06-01 00:00:12.1780,2020-06-01 00:03:58.8640,3255.0,8 Ave & W 31 St,40.750585,-73.994685,505.0,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,1969,0
4,1437,2020-06-01 00:00:21.2550,2020-06-01 00:24:18.9650,367.0,E 53 St & Lexington Ave,40.758281,-73.970694,497.0,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,1997,2


In [19]:
# Splitting Gender (0, male is 1, and female is 2) & Usertype based on Citi Bike's documentation and in preparation for aggregation.
combined_csv["gender_unknown"] = combined_csv["gender"] == 0
combined_csv["gender_unknown"] = combined_csv["gender_unknown"].astype(int)

combined_csv["gender_male"] = combined_csv["gender"] == 1
combined_csv["gender_male"] = combined_csv["gender_male"].astype(int)

combined_csv["gender_female"] = combined_csv["gender"] == 2
combined_csv["gender_female"] = combined_csv["gender_female"].astype(int)

combined_csv["subscriber"] = combined_csv["usertype"] == "Subscriber"
combined_csv["subscriber"] = combined_csv["subscriber"].astype(int)

combined_csv["customer"] = combined_csv["usertype"] == "Customer"
combined_csv["customer"] = combined_csv["customer"].astype(int)



,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,gender_unknown,gender_male,gender_female,subscriber,customer
0,1062,2020-06-01 00:00:03.3720,2020-06-01 00:17:46.2080,3419.0,Douglass St & 4 Ave,40.679279,-73.981540,3419.0,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,1997,2,0,0,1,0,1
1,3810,2020-06-01 00:00:03.5530,2020-06-01 01:03:33.9360,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336.0,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,1969,0,1,0,0,1,0
2,1017,2020-06-01 00:00:09.6140,2020-06-01 00:17:06.8330,389.0,Broadway & Berry St,40.710446,-73.965251,3562.0,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,1988,2,0,0,1,0,1
3,226,2020-06-01 00:00:12.1780,2020-06-01 00:03:58.8640,3255.0,8 Ave & W 31 St,40.750585,-73.994685,505.0,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,1969,0,1,0,0,0,1
4,1437,2020-06-01 00:00:21.2550,2020-06-01 00:24:18.9650,367.0,E 53 St & Lexington Ave,40.758281,-73.970694,497.0,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,1997,2,0,0,1,0,1


In [26]:
# Groupby to get aggregate data for all start stations for the month.
start_station_df = combined_csv.groupby(["start station id", "start station name", "start station latitude", "start station longitude"]).sum()
# Only grab necessary columns.
start_station_df = start_station_df[["gender_male", "gender_female", "gender_unknown", "subscriber", "customer"]]
# Reset the index for cleanliness.
start_station_df.reset_index(inplace=True)
# Needed a total uses column.
start_station_df["total_uses"] = start_station_df["subscriber"] + start_station_df["customer"]
# stationtype 1 is for the starting stations.  Used for filtering in Tableau.
start_station_df["stationtype"] = 1
# Rename columns.
start_station_df.columns = ["station_id", "station_name", "station_latitude", "station_longitude", "gender_male", "gender_female", "gender_unknown", "subscriber", "customer", "total_uses", "station_type"]
# Preview result.
start_station_df.head()

,station_id,station_name,station_latitude,station_longitude,gender_male,gender_female,gender_unknown,subscriber,customer,total_uses,station_type
0,72.0,W 52 St & 11 Ave,40.767000,-73.995000,0,1,0,1,0,1,1
1,72.0,W 52 St & 11 Ave,40.767272,-73.993929,34073,11845,4378,39034,11262,50296,1
2,79.0,Franklin St & W Broadway,40.719000,-74.007000,3,0,0,2,1,3,1
3,79.0,Franklin St & W Broadway,40.719116,-74.006667,15968,5749,2028,19822,3923,23745,1
4,82.0,St James Pl & Pearl St,40.710000,-74.001000,0,1,0,1,0,1,1


In [28]:
start_station_df.drop_duplicates(subset ="station_id", 
                     keep = False, inplace = True)

In [30]:
start_station_df.head()

,station_id,station_name,station_latitude,station_longitude,gender_male,gender_female,gender_unknown,subscriber,customer,total_uses,station_type
6,83.0,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,13175,5128,2464,16095,4672,20767,1
9,119.0,Park Ave & St Edwards St,40.696089,-73.978034,4665,1914,565,6262,882,7144,1
10,120.0,Lexington Ave & Classon Ave,40.686768,-73.959282,7348,4153,1130,10362,2269,12631,1
11,127.0,Barrow St & Hudson St,40.731724,-74.006744,41060,14681,4146,51174,8713,59887,1
12,128.0,MacDougal St & Prince St,40.727103,-74.002971,41661,19102,4598,55114,10247,65361,1


In [ ]:
# Groupby to get aggregate data for all start stations for the month.
end_station_df = df.groupby(["end station id", "end station name", "end station latitude", "end station longitude"]).sum()
# Only grab necessary columns.
end_station_df = end_station_df[["gender_male", "gender_female", "gender_unknown", "subscriber", "customer"]]
# Reset the index for cleanliness.
end_station_df.reset_index(inplace=True)
# Needed a total uses column.
end_station_df["total_uses"] = end_station_df["subscriber"] + end_station_df["customer"]
# stationtype 0 is for the ending stations.  Used for filtering in Tableau.
end_station_df["stationtype"] = 0
# Rename columns.
end_station_df.columns = ["station_id", "station_name", "station_latitude", "station_longitude", "gender_male", "gender_female", "gender_unknown", "subscriber", "customer", "total_uses", "station_type"]
# Preview result.
end_station_df.head()